In [1]:
# -*- coding:utf-8 -*-
from __future__ import division
import os,sys

%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

In [3]:
import pandas as pd
from sqlalchemy import create_engine

engine=create_engine('mysql+pymysql://root:@127.0.0.1:3306/law?charset=utf8')
sql=pd.read_sql('all_gzdata',engine,chunksize=10000)

In [5]:
counts=[i['fullURLId'].value_counts() for i in sql]
counts=pd.concat(counts).groupby(level=0).sum()
print(counts)

101001       1500
101002       2190
101003     101841
101004         35
101005         15
101006         24
101007         39
101008        107
101009        246
102001        454
102002       4012
102003        287
102004         61
102005         73
102006         50
102007        157
102008         75
102009         39
103002         83
103003        543
106001       1199
107001      49359
1999001     53807
301001       4693
Name: fullURLId, dtype: int64


In [7]:
counts=counts.reset_index()
counts.columns=['index','num']
print(counts)
counts['type']=counts['index'].str.extract('(\d{3})',expand=True)
print(counts.head())
counts_=counts[['type','num']].groupby('type').sum()
print(counts_)
counts_.sort_values(by='num',ascending=False)
print(counts_['num'].sum())

      index     num
0    101001    1500
1    101002    2190
2    101003  101841
3    101004      35
4    101005      15
5    101006      24
6    101007      39
7    101008     107
8    101009     246
9    102001     454
10   102002    4012
11   102003     287
12   102004      61
13   102005      73
14   102006      50
15   102007     157
16   102008      75
17   102009      39
18   103002      83
19   103003     543
20   106001    1199
21   107001   49359
22  1999001   53807
23   301001    4693
    index     num type
0  101001    1500  101
1  101002    2190  101
2  101003  101841  101
3  101004      35  101
4  101005      15  101
         num
type        
101   105997
102     5208
103      626
106     1199
107    49359
199    53807
301     4693
220889


In [8]:
print(counts.head())
print(counts_.head())

    index     num type
0  101001    1500  101
1  101002    2190  101
2  101003  101841  101
3  101004      35  101
4  101005      15  101
         num
type        
101   105997
102     5208
103      626
106     1199
107    49359


In [13]:
def count107(i):
    j=i[['fullURL']][i['fullURLId'].str.contains('107')].copy()
    j['type']=None
    j['type'][j['fullURL'].str.contains('info/.+?/')]=u'知识首页'
    j['type'][j['fullURL'].str.contains('info/.+?/.+?')]=u'知识列表页'
    j['type'][j['fullURL'].str.contains('/\d+?_*\d+?\.html')]=u'知识内容页'
    return j['type'].value_counts()

In [15]:
import pandas as pd
from sqlalchemy import create_engine

engine=create_engine('mysql+pymysql://root:@127.0.0.1:3306/law?charset=utf8')
sql=pd.read_sql('all_gzdata',engine,chunksize=10000)

counts2=[count107(i) for i in sql]
counts2=pd.concat(counts2).groupby(level=0).sum()
print(counts2)

知识内容页    46808
知识列表页     2584
知识首页      2265
Name: type, dtype: int64


In [17]:
import pandas as pd 
from sqlalchemy import create_engine
engine=create_engine('mysql+pymysql://root:@127.0.0.1:3306/law?charset=utf8')
sql=pd.read_sql('all_gzdata',engine,chunksize=10000)

c=[i['realIP'].value_counts() for i in sql]
counts3=pd.concat(c).groupby(level=0).sum()
counts3=pd.DataFrame(counts3)
print(len(counts3))
print(counts3['realIP'].sum())
counts3[1]=1
counts3.groupby('realIP').sum()

c3_=counts3.groupby('realIP').sum()
c3_.head()

77526
230889


,1
realIP,
1,48201
2,14334
3,5175
4,2872
5,1641


In [20]:
import pandas as pd
from sqlalchemy import create_engine
engine=create_engine('mysql+pymysql://root:@127.0.0.1:3306/law?charset=utf8')
sql=pd.read_sql('all_gzdata',engine,chunksize=10000)

for i in sql:
    d=i[['realIP','fullURL']]
    d=d[d['fullURL'].str.contains('\.html')].copy()
    d.to_sql('cleaned_gzdata',engine,index=False,if_exists='append')
    

F:\Anaconda\anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE9\\xAA\\x9E\\xE5\\xAD\\x98...' for column 'fullURL' at row 7678")
  result = self._query(query)


In [21]:
import pandas as pd
from sqlalchemy import create_engine

engine=create_engine('mysql+pymysql://root:@127.0.0.1:3306/law?charset=utf8')
sql=pd.read_sql('cleaned_gzdata',engine,chunksize=10000)
for i in sql:
    d=i.copy()
    d['fullURL']=d['fullURL'].str.replace('_\d{0,2}.html','.html')
    d=d.drop_duplicates()
    d.to_sql('changed_gzdata',engine,index=False,if_exists='append')
    

In [22]:
import pandas as pd
from sqlalchemy import create_engine

engine=create_engine('mysql+pymysql://root:@127.0.0.1:3306/law?charset=utf8')
sql=pd.read_sql('cleaned_gzdata',engine,chunksize=10000)

for i in sql:
    d=i.copy()
    d['type_1']=d['fullURL']
    d['type_1'][d['fullURL'].str.contains('(ask)|(askzt)')]='zixun'
    d.to_sql('splited_gzdata',engine,index=False,if_exists='append')

F:\Anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.
F:\Anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [23]:
import numpy as np
from numpy import linalg as la

def ecludSim(a,b):
    return 1.0/(1.0+la.norm(a-b))

def pearsSim(a,b):
    if len(a)<3:return 1.0
    return 0.5+0.5*corrcoef(a,b,rowvar=0)[0][1]

def consSim(a,b):
    num=float(a.T*b)
    denum=la.norm(a)*la.norm(b)
    return 0.5+0.5*(num/denum)

def Jaccard(a,b):
    return 1.0*(a*b).sum()/(a+b-a*b).sum()